In [7]:
import os
import sys
import pandas as pd
import yaml 
from matplotlib import pyplot as plt
from matplotlib import ticker as mticker
import statsmodels.api as sm
import numpy as np
from itertools import product
import subprocess

with open("../../config.yaml.local", "r") as f:
    LOCAL_CONFIG = yaml.safe_load(f)
#with open("../../config.yaml", "r") as f:
#    CONFIG = yaml.safe_load(f)
sys.path.append("../python")

import globals
import data_tools as dt
import writing_tools as wt

LOCAL_PATH = LOCAL_CONFIG["LOCAL_PATH"]
RAW_DATA_PATH = LOCAL_CONFIG["RAW_DATA_PATH"]
DATA_PATH = LOCAL_CONFIG["DATA_PATH"]
R_PATH = LOCAL_CONFIG["R_PATH"]

RUN_R_SCRIPTS = False


In [8]:
df = dt.get_user_by_week_panel(overwrite=False)
weekly_prices = dt.get_price_weekly()

c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:89: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  return x.dt.to_period('W-SAT').dt.start_time


In [9]:
weekly_prices['wow_growth'] = np.log(weekly_prices['btc_price']) - np.log(weekly_prices['btc_price'].shift(1))
weekly_prices['mom_growth'] = np.log(weekly_prices['btc_price']) - np.log(weekly_prices['btc_price'].shift(4))
weekly_prices['mom2_growth'] = np.log(weekly_prices['btc_price']) - np.log(weekly_prices['btc_price'].shift(8))
weekly_prices['yoy_growth'] = np.log(weekly_prices['btc_price']) - np.log(weekly_prices['btc_price'].shift(52))

In [10]:
# merge on weekly prices
df = df.merge(weekly_prices, on='week', how='left')

# remove SN employees
df = df.loc[~df['userId'].isin(globals.sn_employee_ids)].reset_index(drop=True)

# remove anon
df = df.loc[df['userId'] != globals.anon_id].reset_index(drop=True)

# remove territory owners
tdf = dt.get_territory_transfers()
df = df.loc[~df['userId'].isin(tdf['userIdTo'].unique())].reset_index(drop=True)

# remove weeks in which user is not active 
df['inactive'] = (df['weeks_since_last_activity']>=1) & (df['length_of_inactivity']>=4)
df['became_inactive'] = (df['weeks_since_last_activity']==1) & (df['length_of_inactivity']>=4)
df = df.loc[(~df['inactive']) | (df['became_inactive'])].reset_index(drop=True)

# output
df.to_parquet(os.path.join(DATA_PATH, "profitability-analysis.parquet"), index=False)

In [11]:
wt.update_results({
    "InactiveRateBaseline": f"{df['became_inactive'].mean()*100:.1f}"
})

{'PostsCostElasticity': '-0.265',
 'PostsCostDoublingEffect': '16.8',
 'Zaps48CostElasticity': '0.219',
 'Zaps48CostDoublingEffect': '16.4',
 'Comments48CostElasticity': '0.043',
 'Comments48CostDoublingEffect': '3.0',
 'NumItems': '1,121,485',
 'NumPosts': '207,219',
 'NumComments': '914,266',
 'NumUsers': '11,228',
 'NumTerritories': '117',
 'StartDate': 'June 11, 2021',
 'EndDate': 'October 5, 2025',
 'TotalZaps': '146 million',
 'NonCustodialDate': 'January 3, 2025',
 'TotalCost': '13 million',
 'Sats48': '94',
 'Comments48': '90',
 'NumPostingFeeChanges': '199',
 'AvgPostingCost': '51',
 'AvgSats48': '364',
 'AvgComments48': '3.3',
 'SdPostingCost': '241',
 'SdSats48': '6,662',
 'SdComments48': '9.6',
 'AvgPostingCostCents': '5',
 'NumberHighQuality': '44,682',
 'NumberLowQuality': '146,652',
 'HighQualityEffect': '7.1',
 'ShareHighQuality': '23.4',
 'InactiveRateBaseline': '14.1'}

In [12]:
res = subprocess.run([R_PATH, LOCAL_PATH + "/src/R/profitability-analysis.R"], check=True, capture_output=True, text=True)
print(res.stdout)


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Fri, Dec 12, 2025 - 4:34:13 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{4}{c}{\textit{Dependent variable:}} \\ 
\cline{2-5} 
\\[-1.8ex] & \multicolumn{4}{c}{became\_inactive} \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4)\\ 
\hline \\[-1.8ex] 
 Unprofitable in last 8 weeks & 0.105$^{***}$ & 0.062$^{***}$ & 0.060$^{***}$ & 0.055$^{***}$ \\ 
  & (0.003) & (0.003) & (0.003) & (0.003) \\ 
  & & & & \\ 
 $\ldots$ $\times$ BTC price appreciation in last 8 weeks &  &  &  & 0.056$^{***}$ \\ 
  &  &  &  & (0.014) \\ 
  & & & & \\ 
 log(Items posted in last 8 weeks) &  & $-$0.029$^{***}$ & $-$0.029$^{***}$ & $-$0.029$^{***}$ \\ 
  &  & (0.001) & (0.001) & (0.001) \\ 
  & & & & \\ 
 Constant & 0.079$^{***}$ & 0.157$^{***}$ &  &  \\ 
  & (0.001) & (0.002) &  &  \\

In [17]:
coefs = pd.read_csv(
    os.path.join(DATA_PATH, 'profitability_analysis.csv'),
    index_col = 0
)

coef1 = coefs.loc['unprofitableTRUE', 'coef(r4)']
coef2 = coefs.loc['unprofitable_X_mom2_growth', 'coef(r4)']

wt.update_results({
    'UnprofitableExitEffect': f"{coef1*100:.1f}", 
    'UnprofitableExitBTCGrowthEffect': f"{(coef1 + coef2*0.1)*100:.1f}"
})

#coef = coefs.loc['hi_quality_share', 'coef(r4)']

#wt.update_results({
#    'HighQualityEffect': f"{(coef*0.5)*100:.1f}"
#})

{'PostsCostElasticity': '-0.265',
 'PostsCostDoublingEffect': '16.8',
 'Zaps48CostElasticity': '0.219',
 'Zaps48CostDoublingEffect': '16.4',
 'Comments48CostElasticity': '0.043',
 'Comments48CostDoublingEffect': '3.0',
 'NumItems': '1,121,485',
 'NumPosts': '207,219',
 'NumComments': '914,266',
 'NumUsers': '11,228',
 'NumTerritories': '117',
 'StartDate': 'June 11, 2021',
 'EndDate': 'October 5, 2025',
 'TotalZaps': '146 million',
 'NonCustodialDate': 'January 3, 2025',
 'TotalCost': '13 million',
 'Sats48': '94',
 'Comments48': '90',
 'NumPostingFeeChanges': '199',
 'AvgPostingCost': '51',
 'AvgSats48': '364',
 'AvgComments48': '3.3',
 'SdPostingCost': '241',
 'SdSats48': '6,662',
 'SdComments48': '9.6',
 'AvgPostingCostCents': '5',
 'NumberHighQuality': '44,682',
 'NumberLowQuality': '146,652',
 'HighQualityEffect': '7.1',
 'ShareHighQuality': '23.4',
 'InactiveRateBaseline': '14.1',
 'UnprofitableExitEffect': '5.5',
 'UnprofitableExitBTCGrowthEffect': '6.1'}

In [15]:
coefs

,coef(r4)
unprofitableTRUE,0.055010
unprofitable_X_mom2_growth,0.056069
log_items,-0.029039
